# Tuning models optimization

##### Modules

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import dill
import datetime
import math
import itertools

from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
import statsmodels.api

from sklearn import model_selection
from sklearn.metrics import mean_squared_error, classification_report, roc_curve, auc, plot_confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
import xgboost as xgb

%matplotlib inline

##### Define workspace

In [2]:
os.chdir('C:\\Users\\Megaport\\Desktop\\jupyterNotebook')
os.getcwd()

'C:\\Users\\Megaport\\Desktop\\jupyterNotebook'

##### Import

In [3]:
# dfPoolMLCCA = pd.read_pickle('D:\\jupyterDatasets\\20221031_table_dfPoolMLCCA.csv')
feature_matrix = pd.read_pickle('D:\\jupyterDatasets\\20221112_table_feature_matrix.csv')
target = pd.read_pickle('D:\\jupyterDatasets\\20221119_table_target.csv')

# print(dfPoolMLCCA.shape)
print(feature_matrix.shape)
print(target.shape)

(836553, 70)
(836553,)


In [4]:
feature_matrix.columns

Index(['choc_cote', 'ageMeanConductors', 'nbVeh', 'prof_2.0', 'prof_3.0',
       'planGrp_1.0', 'surf_2.0', 'surf_8.0', 'atm_2.0', 'atm_3.0', 'atm_5.0',
       'atm_7.0', 'atm_8.0', 'vospGrp_1.0', 'catv_EPD_exist_1',
       'catv_PL_exist_1', 'trajet_coursesPromenade_conductor_1',
       'sexe_male_conductor_1', 'sexe_female_conductor_1',
       'intGrp_Croisement circulaire', 'intGrp_Croisement de deux routes',
       'intGrp_Hors intersection', 'intGrp_Passage à niveau',
       'catv_train_exist_1', 'infra_3.0', 'infra_5.0', 'infra_7.0',
       'infra_9.0', 'catr_2.0', 'catr_3.0', 'catr_4.0', 'catr_9.0',
       'hourGrp_nuit', 'lum_2.0', 'lum_3.0', 'lum_5.0', 'circ_2.0', 'circ_3.0',
       'circ_4.0', 'nbvGrp_1', 'nbvGrp_2', 'nbvGrp_3', 'nbvGrp_4+',
       'catv_2_roues_exist_1', 'col_2.0', 'col_3.0', 'col_4.0', 'col_5.0',
       'col_6.0', 'col_7.0', 'obsGrp_Pas d'Obstacle', 'situ_2.0', 'situ_3.0',
       'situ_4.0', 'situ_6.0', 'situ_8.0', 'populationGrp_Grande Ville',
       'popu

##### Splitting into train/test sets

In [5]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(feature_matrix, target, test_size=0.2, random_state=1)

In [6]:
train = xgb.DMatrix(data=X_train, label=y_train)
test = xgb.DMatrix(data=X_test, label=y_test)

In [8]:
print('Prevalence y train:', round(sum(y_train) / len(y_train), 4))
print('Prevalence y test:', round(sum(y_test) / len(y_test), 4))

Prevalence y train: 0.4234
Prevalence y test: 0.423


### Models

### 1- XGBoost
**Les paramètres généraux**  
- booster : Le type de booster utilisé (par défaut gbtree).  
- nthread : Le nombre de coeurs à utiliser pour le calcul parallèle (par défaut tous les coeurs disponibles sont utilisés).  

**Les paramètres du booster (on se limitera ici au cas des arbres)**  
- num_boost_round : Le nombre maximum d'itérations/d'arbres construits (vaut 100 par défaut).  
- learning_rate : Contrôle le 'taux d’apprentissage'. A chaque étape de boosting, on introduit une constante dans la formule de mise à jour des modèles. Elle réduit le poids obtenu par rapport aux performances pour prévenir l'overfitting. Une valeur faible entraîne un modèle plus robuste au sur-apprentissage, mais un calcul et une convergence plus lents. Pensez à augmenter le nombre d'arbres lorsque learning_rate est faible (vaut 0.3 par défaut, et doit être compris entre 0 et 1).  
- min_split_loss : Réduction de perte minimale requise pour effectuer une partition supplémentaire sur un nœud de l'arbre. Plus il est grand, plus l'algorithme sera conservateur.  
- max_depth : Contrôle la profondeur des arbres. Plus les arbres sont profonds, plus le modèle est complèxe et plus grandes sont les chances d'overfitting (vaut 6 par défaut).  
- min_child_weight : Critère d'arrêt relatif à la taille minimum du nombre d'observation dans un noeud (vaut 1 par défaut).  
- subsample : Permet d'utiliser un sous-échantillon du dataset d'entraînement pour chaque arbre (vaut 1 par défaut, pas de sous-échantillonnage ; et doit être compris entre 0 et 1).  
- colsample_bytree : Permet d'utiliser un certain nombre de variables parmi celles d'origine (vaut 1 par défaut, toutes les variables sont séléctionnées ; et doit être compris entre 0 et 1).  
- reg_lambda et reg_alpha : contrôlent respectivement la régularisation L1 et L2 sur les poids (équivalent aux régression Ridge et Lasso).  

- Fonction objectif à utiliser
    - binary:logistic pour la classification binaire. Retourne les probabilités pour chaque classe.
    - reg:linear pour la régression.
    - multi:softmax pour la classification multiple en utilisant la fonction softmax. Retourne les labels prédits.
    - multi:softprob pour la classification multiple en utilisant la fonction softmax. Retourne les probabilités pour chaque classe.
- eval_metric : Métrique d'évaluation (par défaut l'erreur de prédiction pour la classification, le RMSE pour la régression).
Les métriques disponibles sont : mae (Mean Absolute Error), Logloss, AUC, RMSE, error mologloss, etc...
- early_stopping_rounds : pour arrêter l'apprentissage quand l'évaluation sur l'ensemble de test ne s'améliore plus durant un certain nombre d'itérations. L'erreur de validation doit diminuer au moins tous les early_stopping_rounds pour continuer l'entraînement.

##### Initial model

In [17]:
### Initiating parameters for basic XGBoost
# params = {'objective' : 'binary:logistic', 
#           'booster' : 'gbtree', 
#           'learning_rate' : 1,
#           'eval_metric' : 'AUC'}
params = [
    ('objective', 'binary:logistic'),
    ('max_depth', 6),
    ('eval_metric', 'auc'),
    ('early_stopping_rounds', 10)
]

### Basic XGBoost without tuning
clf_xgb = xgb.train(params=params, dtrain=train, 
                    num_boost_round=50, 
                    evals=[(train, 'train'), (test, 'eval')])

[14:50:14] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "early_stopping_rounds" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.75326	eval-auc:0.75193
[1]	train-auc:0.75916	eval-auc:0.75808
[2]	train-auc:0.76525	eval-auc:0.76358
[3]	train-auc:0.76890	eval-auc:0.76710
[4]	train-auc:0.77253	eval-auc:0.77078
[5]	train-auc:0.77496	eval-auc:0.77323
[6]	train-auc:0.77653	eval-auc:0.77470
[7]	train-auc:0.77926	eval-auc:0.77725
[8]	train-auc:0.78084	eval-auc:0.77874
[9]	train-auc:0.78223	eval-auc:0.78000
[10]	train-auc:0.78386	eval-auc:0.78153
[11]	train-auc:0.78490	eval-auc:0.78252
[12]	train-auc:0.78554	eval-auc:0.78300
[13]	train-auc:0.78610	eval-auc:0.78349
[14]	train-auc:0.786

In [47]:
xgb_pred_train = clf_xgb.predict(X_train)
xgb_pred_test = clf_xgb.predict(X_test)

In [48]:
pd.crosstab(y_train, xgb_pred_train, colnames=['xgb_pred_train'], normalize=True)

xgb_pred_train,0,1
gravGrp_2_34,,
0,0.488529,0.088071
1,0.168202,0.255198


In [49]:
print(classification_report(y_train, xgb_pred_train))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79    385885
           1       0.74      0.60      0.67    283357

    accuracy                           0.74    669242
   macro avg       0.74      0.72      0.73    669242
weighted avg       0.74      0.74      0.74    669242



In [50]:
pd.crosstab(y_test, xgb_pred_test, colnames=['xgb_pred_test'], normalize=True)

xgb_pred_test,0,1
gravGrp_2_34,,
0,0.485892,0.091094
1,0.171214,0.251801


In [51]:
print(classification_report(y_test, xgb_pred_test))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79     96536
           1       0.73      0.60      0.66     70775

    accuracy                           0.74    167311
   macro avg       0.74      0.72      0.72    167311
weighted avg       0.74      0.74      0.73    167311



In [55]:
# AUC train
fpr_xgb_train, tpr_xgb_train, seuils = roc_curve(y_train, xgb_pred_train, pos_label=1)
roc_auc_xgb_train = auc(fpr_xgb_train, tpr_xgb_train)
# AUC test
fpr_xgb_test, tpr_xgb_test, seuils = roc_curve(y_test, xgb_pred_test, pos_label=1)
roc_auc_xgb_test = auc(fpr_xgb_test, tpr_xgb_test)

print(roc_auc_xgb_train)
print(roc_auc_xgb_test)

0.724995981244733
0.7186868174680067


In [ ]:
# ROC curve
plt.figure(figsize=(4, 4))
plt.plot(fpr_xgb_test, tpr_xgb_test, color='orange', lw=2, label=round(roc_auc_xgb_test, 2))
plt.plot(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01), 'b--', label='0.50')
plt.ylabel('Taux vrais positifs')
plt.xlabel('Taux faux positifs')
plt.title('Courbe ROC')
plt.legend(loc='lower right');

##### Tuning

In [57]:
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01],
    'gamma': [0, 0.25, 1.0],
    'reg_lambda': [0, 1.0, 10.0],
    'scale_pos_weight': [1, 3, 5]
}

In [60]:
optimal_params = GridSearchCV(
    estimator=clf_xgb,
    param_grid=param_grid,
    scoring='roc_auc',
    n_jobs=10,
    cv=3)

In [ ]:
optimal_params.fit(X_train, 
                   y_train,
                   eval_metric='auc',
                   eval_set=[(X_test, y_test)])

In [ ]:
print(optimal_params.best_params_)

In [45]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

estimator = XGBClassifier(
    objective= 'binary:logistic',
    num_boost_round=50,
    seed=1
)

parameters = {
    'max_depth': [2, 3, 4]
}

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = -1,
    cv = 5,
    verbose=True
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[16:14:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_boost_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
    

In [46]:
grid_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=4, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_boost_round=50, num_parallel_tree=1,
              predictor='auto', random_state=1, reg_alpha=0, ...)

In [51]:
grid_search.cv_results_

{'mean_fit_time': array([181.85422401, 323.30103192,  67.18678126]),
 'std_fit_time': array([92.71668813, 86.40454585,  6.0970038 ]),
 'mean_score_time': array([4.21838226, 2.55384502, 0.51226511]),
 'std_score_time': array([1.22020777, 2.28113775, 0.08223257]),
 'param_max_depth': masked_array(data=[2, 3, 4],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2}, {'max_depth': 3}, {'max_depth': 4}],
 'split0_test_score': array([0.78250581, 0.78760482, 0.79023297]),
 'split1_test_score': array([0.78419678, 0.78953565, 0.79287195]),
 'split2_test_score': array([0.78079168, 0.78614412, 0.78952315]),
 'split3_test_score': array([0.78241267, 0.787517  , 0.79130971]),
 'split4_test_score': array([0.7812303 , 0.78708282, 0.79095375]),
 'mean_test_score': array([0.78222745, 0.78757688, 0.79097831]),
 'std_test_score': array([0.00118708, 0.00110798, 0.00112866]),
 'rank_test_score': array([3, 2, 1])}